# Short Term Weather Forecasting with Single Shot Method

Import libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data

In [23]:
df=pd.read_csv('../dataset_for_analysis/data_weather_new.csv',parse_dates=['date'])

## Normalization and modeling

Define the features and the targeted variables to be predicted

In [24]:
X=df[['cloud_cover','sunshine','global_radiation','pressure']]
Y=df[['mean_temp','precipitation','min_temp','max_temp']]

Initiate the Standard Scaler

In [25]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

## Support Vector Machine model

### Single output

Define the models for each predicted targeted variable

In [26]:
from sklearn.svm import SVR

# Create linear regression models
models={
    'mean_temp':SVR(),
    'precipitation':SVR(),
    'min_temp':SVR(),
    'max_temp':SVR()
}

Use cross validation method to train and evaluate the models

In [28]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Initiate the Time Series Split
tscv=TimeSeriesSplit(n_splits=5)

# Train the models
for target_variable,model in models.items():

    # Store the mse value
    mse_scores=[]

    # Split the training set and test set
    for train_index,test_index in tscv.split(df):
        X_train=X.iloc[train_index]
        y_train=Y[target_variable].iloc[train_index]

        X_test=X.iloc[test_index]
        y_test=Y[target_variable].iloc[test_index]

        # Normalize the data
        X_train=scaler.fit_transform(X_train)
        X_test=scaler.fit_transform(X_test)

        # Train the model
        model.fit(X_train,y_train)

        # Make the predictions
        y_pred=model.predict(X_test)

        # Calculate the mean squared error
        mse=mean_squared_error(y_test,y_pred)
        mse_scores.append(mse)

    # Print the result
    print(f'MSE of {target_variable} prediction: {np.mean(mse_scores): .2f}')

MSE of mean_temp prediction:  15.05
MSE of precipitation prediction:  12.82
MSE of min_temp prediction:  15.99
MSE of max_temp prediction:  17.57


Predict the data in 2020

In [22]:
# Predict the mean_temp in 2020
data_2020=df[df['date'].dt.year == 2020]
X_2020=data_2020[['cloud_cover','sunshine','global_radiation','pressure']].values
scaled_X_2020=scaler.fit_transform(X_2020)
y_pred_2020_meantemp=models['mean_temp'].predict(scaled_X_2020)
y_pred_2020_precipitation=models['precipitation'].predict(scaled_X_2020)
y_pred_2020_mintemp=models['min_temp'].predict(scaled_X_2020)
y_pred_2020_maxtemp=models['max_temp'].predict(scaled_X_2020)

print(f"MSE of mean_temp: {mean_squared_error(data_2020['mean_temp'],y_pred_2020_meantemp): .2f}")
print(f"MSE of precipitation: {mean_squared_error(data_2020['precipitation'],y_pred_2020_precipitation): .2f}")
print(f"MSE of min_temp: {mean_squared_error(data_2020['min_temp'],y_pred_2020_mintemp): .2f}")
print(f"MSE of max_temp: {mean_squared_error(data_2020['max_temp'],y_pred_2020_maxtemp): .2f}")


MSE of mean_temp:  18.73
MSE of mean_temp:  28.37
MSE of mean_temp:  19.79
MSE of mean_temp:  23.48


Visualize the result

In [ ]:

# Create the figure containing two subplots
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(18,8))

# The plot for mean_temp
ax1.plot(data_2020['date'],data_2020['mean_temp'],label='Actual')
ax1.plot(data_2020['date'],y_pred_2020_meantemp, label='Predicted')
ax1.set_xlabel('Date')
ax1.set_ylabel('Mean Temperature (°C)')
ax1.set_title('Mean_temp prediction')
ax1.legend(loc='upper left')

# The plot for precipitation
ax2.plot(data_2020['date'], data_2020['precipitation'],label='Actual')
ax2.plot(data_2020['date'], y_pred_2020_precipitation,label='Predicted')
ax2.set_xlabel('Date')
ax2.set_ylabel('Precipitation (mm)')
ax2.set_title('Precipitation prediction')
ax2.legend(loc='upper left')

ax1.xaxis.set_major_locator(plt.MaxNLocator(14))
ax2.xaxis.set_major_locator(plt.MaxNLocator(14))

# Adjust the layout of the subplots
plt.tight_layout()
plt.show()

## Multiple output

Train and test the model

In [29]:
from sklearn.multioutput import MultiOutputRegressor

svr=SVR()
model=MultiOutputRegressor(svr)

mse_scores1=[]

for train_index,test_index in tscv.split(df):
    X_train=X.iloc[train_index]
    y_train=Y.iloc[train_index]

    X_test=X.iloc[test_index]
    y_test=Y.iloc[test_index]

    # Normalize the data
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.fit_transform(X_test)

    # Train the model
    model.fit(X_train,y_train)

    # Make the predictions
    y_pred=model.predict(X_test)

    # Calculate the mean squared error
    mse=mean_squared_error(y_test,y_pred)
    mse_scores1.append(mse)

# Print the result
print(f'MSE of prediction: {np.mean(mse_scores1): .2f}')

MSE of prediction:  15.355
